# 一般化割当問題

一対一の割当問題を一対多に拡張した問題設定．
一つの仕事は一つの資源に割り当てられるが，各資源はその許容量の範囲内で複数の仕事を実行して良い．
仕事を実行した時に許容量から消費される資源の量 $a_{ij}$ は資源 $i$ と仕事 $j$ の組み合わせで決まっている．

$$
\mathrm{minimize} \sum_{i, j} c_{ij} x_{ij} \\
\mathrm{subject \ to} \sum_j a_{ij} x_{ij} \le b_i \ \forall i \\
\sum_i x_{ij} = 1 \ \forall j \\
x_{ij} \in \{0,1\}
$$


- $a_{ij}=1, b_i=1$ で通常の割当問題に帰着される
- 一般化割当問題(GAP)はナップサック問題(KP) $\max_x \sum_j p_j x_j, \ \mathrm{s.t.\ } \sum_j w_j x_j \le C, x_j \in \{0,1\}$ を含む．なぜなら，資源$i$の数$m$を2とし，$i=1$に対して$c_{1j} = -p_j, a_{1j}=w_j, b_1=C$と定義して$i=2$に対して$c_{2j} = 0, a_{1j}=0, b_1\ge 0$ (制約を無効化する) と定義すれば資源1をナップサック内，資源2をナップサック外として表現できる ($c_{ij}\ge 0$である必要があれば，コスト全体に大きな定数を足せば良い)

ここではMIPソルバーを用いた求解のみ試す．
(メタヒューリスティクスは実装していない)

問題インスタンスのダウンロードは `get_gap_data.sh` を親ディレクトリから実行せよ．

In [1]:
folder = "../data/gap/"
f = open(folder + "a05100")
data = f.readlines()
f.close()

In [2]:
m, n = list(map(int, data[0].split()))
cost, a, b = {}, {}, {}
data_ = []
for row in data[1:]:
    data_.extend(list(map(int, row.split())))
count = 0
for i in range(m):
    for j in range(n):
        cost[i, j] = data_[count]
        count += 1
for i in range(m):
    for j in range(n):
        a[i, j] = data_[count]
        count += 1
for i in range(m):
    b[i] = data_[count]
    count += 1

In [3]:
from pyscipopt import Model, quicksum

I = list(range(m))
J = list(range(n))
model = Model("gap")
x = {}
for i in I:
    for j in J:
        x[i, j] = model.addVar(vtype="B", name=f"x[{i},{j}]")

for i in I:
    model.addCons(quicksum(a[i, j] * x[i, j] for j in J) <= b[i])
for j in J:
    model.addCons(quicksum(x[i, j] for i in I) == 1)

model.setObjective(quicksum(cost[i, j] * x[i, j] for i in I for j in J), sense='minimize')

model.optimize()

presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 105 upgd conss, 0 impls, 100 clqs
   (0.0s) probing: 51/500 (10.2%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (0.0s) probing aborted: 50/50 successive totally useless probings
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 100 cliques
presolved problem has 500 variables (500 bin, 0 int, 0 impl, 0 cont) and 105 constraints
      5 constraints of type <knapsack>
    100 constraints of type <setppc>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |ro

In [4]:
print(model.getObjVal())

1698.0
